# Homework12

Exercises with Neural Networks

## Goals

- Get familiar with neural network setup, design, data preparation and training process
- Practice setting up the ingredients and parameters for the training loop
- Experiment with neural networks for classification tasks


### Setup

Run the following 2 cells to import all necessary libraries and helpers for this homework

In [ ]:
!wget -q https://github.com/PSAM-5020-2025S-A/5020-utils/raw/main/src/data_utils.py
!wget -q https://github.com/PSAM-5020-2025S-A/5020-utils/raw/main/src/nn_utils.py
!wget -qO- https://github.com/PSAM-5020-2025S-A/5020-utils/releases/latest/download/lfw.tar.gz | tar xz

In [ ]:
import torch

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from torch import nn, Tensor

from data_utils import classification_error, display_confusion_matrix
from data_utils import LFWUtils

from nn_utils import get_labels

## Neural Networks and Classification

We saw a neural network that performs linear regression in the [WK12](https://github.com/PSAM-5020-2025S-A/WK12) notebook, but, how can we use one for classification ?

The modifications aren't too hard. We first need to have as many output nodes as we have classes in our dataset. If we're trying to classify images into $25$ categories, we need $25$ output neurons.

We also need to use a different kind of loss/cost function that will force the network to only have one activated neuron in its output layer, per input. If we give it data for something that should be classified as class $7$, the neuron that represents class $7$ at the output layer should be activated way more than the others.

### Cross Entropy Loss

The cost of being wrong for a classification network is calculated using a function called *Cross Entropy*. It basically measures how different two distributions are, and in this case, we want to compare the distributions of values from the last layer of our network and what we expected to get at the last layer, over all of the inputs that it was shown.

For example, let's say we are training a classification neural network on a dataset with $3$ classes. The correct distribution of labels in the training dataset is:

| $\text{class}$ | $\text{distribution}$ |
|----|----|
| $0$ | $45\%$ |
| $1$ | $35\%$ |
| $2$ | $20\%$ |

but instead we get the following distribution for the predictions:

| $\text{class}$ | $\text{distribution}$ |
|----|----|
| $0$ | $32\%$ |
| $1$ | $32\%$ |
| $2$ | $36\%$ |

*Cross Entropy* is a way to quantify how different these two distributions are and, hopefully, guide our network parameters to make them more similar.

The actual cost calculation is a sum of how different each prediction is from a "perfect" prediction. For example, if we're trying to classify something as class $0$, we should get output neuron activations that are something like: `[1.0, 0.0, 0.0]`. If instead we get: `[0.5, 0.8, 0.1]`, the *Cross Entropy* contribution for this sample is calculated as:
$$\displaystyle - \log\left(\frac{e^{0.5}}{e^{0.5} + e^{0.8} + e^{0.1}}\right)$$

By using this as the cost function, the neural network can optimize its parameters to increase the value of the first output, while decreasing the values of the other outputs. It doesn't have to get `[1.0, 0.0, 0.0]` exactly, so the network optimizes its parameters to make the first output as different as possible from the others.

Instead of writing our own cost function like we did with the euclidean cost for linear regression, we'll use the built in `CrossEntropyLoss` function in the `PyTorch` library.

### The Process ...

... is the same.

### Start with the Data

As always, we start with the data, and this time put the pixel and label information straight into `Tensor` objects.

In [ ]:
train, test = LFWUtils.train_test_split(0.5)

x_train = Tensor(train["pixels"])
y_train = Tensor(train["labels"]).long()

x_test = Tensor(test["pixels"])
y_test = Tensor(test["labels"]).long()

len(x_train), len(x_test)

### Let's train !

Let's create a single layer neural network, like the one from class, and train it with the training data.

In addition to the actual model/network, we also need an optimizer and a loss function.

In [ ]:
# TODO: Create the model and optimizer, the loss function is already defined

loss_fn = nn.CrossEntropyLoss()

### The Loop

Create a training loop like we saw in class.

This loop should:
- Predict classes by feeding all of the inputs into the `model`
- Measure `loss` (this is just `loss_fn(predicted, actual)`)
- Get the optimizer to compute gradients
- Update parameters

The loop should be repeated as long as the loss keeps improving, and it doesn't look like the model is overfitting with the training data.

In order to check if the model is overfitting, we can sporadically run evaluations within the training loop in order to see if the model performs similarly with `train` and `test` data.

But ! Our network actually outputs a series of values for each image that we give it. In order to determine the exact class number of its predictions, we have to find the index of the output neuron with the largest value, which is an operation called `argmax()` (similar to `argsort()` from week 10).

It's not hard to do this manually, but we can use the `get_labels(model, inputs)` function inside the `nn_utils` file to run our `model` on all of the data in a given dataset and return the predicted labels for all of the samples.

In [ ]:
# TODO: iterate epochs
  # TODO: iterate batches
    # TODO: predict
    # TODO: measure loss
    # TODO: compute gradient and step optimizer
    # TODO: show progress

### Evaluate

This should be similar to the last error values seen during training, but sometimes it changes a bit...

Not a bad idea to check the accuracy of the model using the `classification_error()` function, and look at some confusion matrices.

In [ ]:
# TODO: classification error for train and test data
# TODO: confusion matrices for train and test data

## Neural Networks and PCA

We are seeing how Neural Networks can be _easy_ to build and explain in generic/abstract terms (a bunch of little operators that perform weighted sums of their inputs), but in reality can be really difficult and opaque to steer.

In theory, a couple of well placed neuron layers, with the right hyperparameters, learning rate, loss function, architecture and a good amount of data, can learn to extract information like polynomial features, clusters or even PCA components. But... that's not always the case and sometimes it's not a bad idea to push/bias/encourage the network to go down a certain path.

One way to do this is to pre-process our inputs and do a bit of feature extraction ourselves.

Let's see if we can improve this face classification network by using PCA information instead of pixel data.

### Add PCA

We're going to repeat the training, but this time our data is going to be scaled and PCA'd before going into the neural network.

So, the data preparation flow should be:
- Scale data for `PCA`
- Perform `PCA`

We need one `StandardScaler()` object and one `PCA()` object.

The `train` data goes through the `fit_transform()` function of these objects, while the `test` data only goes through `transform()`.

For the `PCA`, we can aim for an explained variance of $99\%$. This should reduce the number of features significantly to allow us to experiment with our network architecture.

In [ ]:
# TODO: Scale
# TODO: PCA
# TODO: Tensors

### Repeat

Re-create model, optimizer, loss function, then re-run the training loop and evaluate.

In [ ]:
# TODO: Model, Optimizer and Loss Function
# TODO: Training loop
# TODO: Evaluation

### Interpretation

<span style="color:hotpink;">
So... What happens ?<br>
How does training on the <code>PCA</code> data compare to training on the regular data ?

What else does <code>PCA</code> afford us in this case ? ...<br>
How does adding extra layers in the original network compare to adding extra layers in the <code>PCA</code> network?
</span>

<span style="color:hotpink;">EDIT THIS CELL WITH ANSWER</span>